In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

#   https://kashif-sohail.medium.com/read-files-from-google-cloud-storage-bucket-using-local-pyspark-and-jupyter-notebooks-f8bd43f4b42e

## Build Master

In [2]:
keyfile = "/home/manuel/Projects/DataEngineeringZoomcamp/infra/creds.json"

# Use spark gcs connector
conf = (SparkConf()
        .setMaster("spark://localhost:7077")
        .setAppName("GCSRead")
        .set("spark.jars", "./spark-jars/gcs-connector-hadoop3-latest.jar")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", keyfile)
        .set("spark.sql.legacy.parquet.nanosAsLong", "true")
)

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("google.cloud.auth.service.account.enable", "true")
hadoop_conf.set("google.cloud.auth.service.account.json.keyfile", keyfile)

spark: SparkSession = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()

24/04/12 15:22:58 WARN Utils: Your hostname, manuelpc resolves to a loopback address: 127.0.0.1; using 131.155.193.147 instead (on interface wlo1)
24/04/12 15:22:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/12 15:23:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/12 15:23:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Read DataFrame

In [3]:
path=f"gs://weather_data_de_bucket/Actuele10mindataKNMIstations/2/2024/04/*/*/*/weather_data.parquet"

df = spark.read.parquet(path)

In [5]:
df.count()

332

In [ ]:
df.show(5)

In [9]:
df.schema

StructType([StructField('wsi', StringType(), True), StructField('stationname', StringType(), True), StructField('lat', DoubleType(), True), StructField('lon', DoubleType(), True), StructField('height', DoubleType(), True), StructField('D1H', DoubleType(), True), StructField('dd', DoubleType(), True), StructField('dn', DoubleType(), True), StructField('dr', DoubleType(), True), StructField('dsd', DoubleType(), True), StructField('dx', DoubleType(), True), StructField('ff', DoubleType(), True), StructField('ffs', DoubleType(), True), StructField('fsd', DoubleType(), True), StructField('fx', DoubleType(), True), StructField('fxs', DoubleType(), True), StructField('gff', DoubleType(), True), StructField('gffs', DoubleType(), True), StructField('h', DoubleType(), True), StructField('h1', DoubleType(), True), StructField('h2', DoubleType(), True), StructField('h3', DoubleType(), True), StructField('hc', DoubleType(), True), StructField('hc1', DoubleType(), True), StructField('hc2', DoubleTyp

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, BinaryType

StructType(
    [
        StructField("wsi", StringType(), True),
        StructField("stationname", StringType(), True),
        StructField("lat", DoubleType(), True),
        StructField("lon", DoubleType(), True),
        StructField("height", DoubleType(), True),
        StructField("D1H", DoubleType(), True),
        StructField("dd", DoubleType(), True),
        StructField("dn", DoubleType(), True),
        StructField("dr", DoubleType(), True),
        StructField("dsd", DoubleType(), True),
        StructField("dx", DoubleType(), True),
        StructField("ff", DoubleType(), True),
        StructField("ffs", DoubleType(), True),
        StructField("fsd", DoubleType(), True),
        StructField("fx", DoubleType(), True),
        StructField("fxs", DoubleType(), True),
        StructField("gff", DoubleType(), True),
        StructField("gffs", DoubleType(), True),
        StructField("h", DoubleType(), True),
        StructField("h1", DoubleType(), True),
        StructField("h2", DoubleType(), True),
        StructField("h3", DoubleType(), True),
        StructField("hc", DoubleType(), True),
        StructField("hc1", DoubleType(), True),
        StructField("hc2", DoubleType(), True),
        StructField("hc3", DoubleType(), True),
        StructField("n", DoubleType(), True),
        StructField("n1", DoubleType(), True),
        StructField("n2", DoubleType(), True),
        StructField("n3", DoubleType(), True),
        StructField("nc", DoubleType(), True),
        StructField("nc1", DoubleType(), True),
        StructField("nc2", DoubleType(), True),
        StructField("nc3", DoubleType(), True),
        StructField("p0", DoubleType(), True),
        StructField("pp", DoubleType(), True),
        StructField("pg", DoubleType(), True),
        StructField("pr", DoubleType(), True),
        StructField("ps", DoubleType(), True),
        StructField("pwc", DoubleType(), True),
        StructField("Q1H", DoubleType(), True),
        StructField("Q24H", DoubleType(), True),
        StructField("qg", DoubleType(), True),
        StructField("qgn", DoubleType(), True),
        StructField("qgx", DoubleType(), True),
        StructField("qnh", DoubleType(), True),
        StructField("R12H", DoubleType(), True),
        StructField("R1H", DoubleType(), True),
        StructField("R24H", DoubleType(), True),
        StructField("R6H", DoubleType(), True),
        StructField("rg", DoubleType(), True),
        StructField("rh", DoubleType(), True),
        StructField("rh10", DoubleType(), True),
        StructField("Sav1H", DoubleType(), True),
        StructField("Sax1H", DoubleType(), True),
        StructField("Sax3H", DoubleType(), True),
        StructField("Sax6H", DoubleType(), True),
        StructField("sq", DoubleType(), True),
        StructField("ss", DoubleType(), True),
        StructField("Sx1H", DoubleType(), True),
        StructField("Sx3H", DoubleType(), True),
        StructField("Sx6H", DoubleType(), True),
        StructField("t10", DoubleType(), True),
        StructField("ta", DoubleType(), True),
        StructField("tb", DoubleType(), True),
        StructField("tb1", DoubleType(), True),
        StructField("Tb1n6", DoubleType(), True),
        StructField("Tb1x6", DoubleType(), True),
        StructField("tb2", DoubleType(), True),
        StructField("Tb2n6", DoubleType(), True),
        StructField("Tb2x6", DoubleType(), True),
        StructField("tb3", DoubleType(), True),
        StructField("tb4", DoubleType(), True),
        StructField("tb5", DoubleType(), True),
        StructField("td", DoubleType(), True),
        StructField("td10", DoubleType(), True),
        StructField("tg", DoubleType(), True),
        StructField("tgn", DoubleType(), True),
        StructField("Tgn12", DoubleType(), True),
        StructField("Tgn14", DoubleType(), True),
        StructField("Tgn6", DoubleType(), True),
        StructField("tn", DoubleType(), True),
        StructField("Tn12", DoubleType(), True),
        StructField("Tn14", DoubleType(), True),
        StructField("Tn6", DoubleType(), True),
        StructField("tsd", DoubleType(), True),
        StructField("tx", DoubleType(), True),
        StructField("Tx12", DoubleType(), True),
        StructField("Tx24", DoubleType(), True),
        StructField("Tx6", DoubleType(), True),
        StructField("vv", DoubleType(), True),
        StructField("W10", DoubleType(), True),
        StructField("W10-10", DoubleType(), True),
        StructField("ww", DoubleType(), True),
        StructField("ww-10", DoubleType(), True),
        StructField("zm", DoubleType(), True),
        StructField("iso_dataset", BinaryType(), True),
        StructField("product", BinaryType(), True),
        StructField("projection", BinaryType(), True),
        StructField("nhc", BinaryType(), True),
        StructField("za", BinaryType(), True),
        StructField("station", StringType(), True),
        StructField("time", LongType(), True),
    ]
)

In [19]:
df.select(df.stationname, df.station, (df.iso_dataset.isNull()).alias("iso_dataset")).distinct().show()

+--------------------+-------+-----------+
|         stationname|station|iso_dataset|
+--------------------+-------+-----------+
|        BERKHOUT AWS|  06249|      false|
|               K13-A|  06252|      false|
|            D15-FA-1|  06201|      false|
|       WESTDORPE AWS|  06319|      false|
|           IJMOND WP|  06209|      false|
|    WIJK AAN ZEE AWS|  06257|      false|
|         GILZE RIJEN|  06350|      false|
|AMSTERDAM/SCHIPHO...|  06240|      false|
|          Muiden BTP|  06236|      false|
|       EURO PLATFORM|  06321|      false|
|       MARKNESSE AWS|  06273|      false|
|       Nieuwkoop BTP|  06238|      false|
|         IJMUIDEN WP|  06225|      false|
|           K14-FA-1C|  06204|      false|
| BORSSELE ALFA (BSA)|  06317|      false|
|ROTTERDAM THE HAG...|  06344|      false|
|               AWG-1|  06208|      false|
|              DEELEN|  06275|      false|
|TERSCHELLING HOOR...|  06251|      false|
|          STAVENISSE|  06324|      false|
+----------

## Save to Big Query